In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.colors as mcolors
from matplotlib.patches import Patch
from pathlib import Path
from multiprocessing.dummy import Pool as ThreadPool
from collections import defaultdict
from natsort import natsorted
import tsfresh as tf

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

pd.set_option('max_columns', None)

In [3]:
data_path = Path('data_transformed')
events_names = {0: 'Normal',
                1: 'Abrupt Increase of BSW',
                2: 'Spurious Closure of DHSV',
                3: 'Severe Slugging',
                4: 'Flow Instability',
                5: 'Rapid Productivity Loss',
                6: 'Quick Restriction in PCK',
                7: 'Scaling in PCK',
                8: 'Hydrate in Production Line'
               }
columns = ['P-PDG',
           'P-TPT',
           'T-TPT',
           'P-MON-CKP',
           'T-JUS-CKP',
           'QGL',
           'class']
rare_threshold = 0.01

In [4]:
def class_and_file_generator(data_path, real=False, simulated=False, drawn=False):
    for class_path in data_path.iterdir():
        if class_path.is_dir():
            class_code = int(class_path.stem)
            for instance_path in class_path.iterdir():
                if (instance_path.suffix == '.csv'):
                    if (simulated and instance_path.stem.startswith('SIMULATED')) or \
                       (drawn and instance_path.stem.startswith('DRAWN')) or \
                       (real and (not instance_path.stem.startswith('SIMULATED')) and \
                       (not instance_path.stem.startswith('DRAWN'))):
                        yield class_code, instance_path

In [5]:
real_instances = list(class_and_file_generator(data_path, real=True, simulated=False, drawn=False))
simulated_instances = list(class_and_file_generator(data_path, real=False, simulated=True, drawn=False))

In [6]:
#simulirani primjeri za znacajku 'well' imaju vrijednost -1

def load_instance(instance):
    class_code, instance_path = instance
    try:
        well, instance_id = instance_path.stem.split('_')
        if 'WELL' in well:
            well = well.split('-')[1]
        if 'SIMULATED' in well:
            well = '-1'
        df = pd.read_csv(instance_path, index_col='timestamp', parse_dates=['timestamp'])
        assert (df.columns == columns).all(), "invalid columns in the file {}: {}".format(str(instance_path), str(df.columns.tolist()))
        df['class_code'] = class_code
        df['well'] = well
        df['instance_id'] = instance_id
        df = df[['class_code', 'well', 'instance_id'] + columns]
        return df
    except Exception as e:
        raise Exception('error reading file {}: {}'.format(instance_path, e))
        
def load_instances(instances):
    pool = ThreadPool()
    all_df = []
    try:
        for df in pool.imap_unordered(load_instance, instances):
            all_df.append(df)
    finally:
        pool.terminate()
    return all_df

In [7]:
#ucitavanje stvarnih i simuliranih csv primjera u df-ove

instances = real_instances + simulated_instances
data = load_instances(instances)

In [8]:
#pretvorba identifikatora dogadaja i oznake busotine u integer tipove podataka

for df in data:
    df['instance_id'] = pd.to_numeric(df['instance_id'])
    df['well'] = pd.to_numeric(df['well'])
    
print(df.dtypes)
data[0].head()

class_code       int64
well             int64
instance_id      int64
P-PDG          float64
P-TPT          float64
T-TPT          float64
P-MON-CKP      float64
T-JUS-CKP      float64
QGL            float64
class          float64
dtype: object


,class_code,well,instance_id,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,QGL,class
timestamp,,,,,,,,,,
2017-09-25 01:00:30,3,14,20170925010031,2.342338e+07,1.370330e+07,56.558224,2.935319e+06,36.081306,1.625385,3.0
2017-09-25 01:00:40,3,14,20170925010031,2.342221e+07,1.370014e+07,56.564010,2.989604e+06,36.061686,1.541092,3.0
2017-09-25 01:00:50,3,14,20170925010031,2.342104e+07,1.369791e+07,56.569884,3.042945e+06,36.043620,1.513173,3.0
2017-09-25 01:01:00,3,14,20170925010031,2.342034e+07,1.369600e+07,56.576066,3.068551e+06,36.044957,1.496669,3.0
2017-09-25 01:01:10,3,14,20170925010031,2.341950e+07,1.369353e+07,56.581859,3.050824e+06,36.050174,1.411177,3.0


In [9]:
#koliko je null vrijednosti po znacajci u skupu podataka

x = None
for dataFrame in data:
    if x is None:
        x = dataFrame.isna().sum(axis = 0)
    else:
        x += dataFrame.isna().sum(axis = 0)
print(x)

class_code           0
well                 0
instance_id          0
P-PDG              545
P-TPT              567
T-TPT           581249
P-MON-CKP       112190
T-JUS-CKP       170341
QGL            3717542
class              446
dtype: int64


In [10]:
#pretvorba null vrijednosti u konacne vrijednosti uporabom funkcije impute(...)
#dokumentacija https://tsfresh.readthedocs.io/en/latest/api/tsfresh.utilities.html#tsfresh.utilities.dataframe_functions.impute

import tsfresh as tf
print(data[0].dtypes)
imputed_data = []
for df in data:
    df2 = tf.utilities.dataframe_functions.impute(df.loc[:, df.columns != 'timestamp'])
    imputed_df = df2
    imputed_data.append(imputed_df)

class_code       int64
well             int64
instance_id      int64
P-PDG          float64
P-TPT          float64
T-TPT          float64
P-MON-CKP      float64
T-JUS-CKP      float64
QGL            float64
class          float64
dtype: object


/home/marko/miniconda3/lib/python3.8/site-packages/tsfresh/utilities/dataframe_functions.py:171: RuntimeWarning: The columns ['T-JUS-CKP'] did not have any finite values. Filling with zeros.
  warnings.warn("The columns {} did not have any finite values. Filling with zeros.".format(
/home/marko/miniconda3/lib/python3.8/site-packages/tsfresh/utilities/dataframe_functions.py:171: RuntimeWarning: The columns ['QGL'] did not have any finite values. Filling with zeros.
  warnings.warn("The columns {} did not have any finite values. Filling with zeros.".format(
/home/marko/miniconda3/lib/python3.8/site-packages/tsfresh/utilities/dataframe_functions.py:171: RuntimeWarning: The columns ['T-JUS-CKP' 'QGL'] did not have any finite values. Filling with zeros.
  warnings.warn("The columns {} did not have any finite values. Filling with zeros.".format(
/home/marko/miniconda3/lib/python3.8/site-packages/tsfresh/utilities/dataframe_functions.py:171: RuntimeWarning: The columns ['P-MON-CKP' 'T-JUS-CKP

In [13]:
#provjera jesu li uklonjene sve null vrijednosti

x = None
for df in imputed_data:
    if x is None:
        x = df.isna().sum(axis = 0)
    else:
        x += df.isna().sum(axis = 0)
print(x)

class_code     0
well           0
instance_id    0
P-PDG          0
P-TPT          0
T-TPT          0
P-MON-CKP      0
T-JUS-CKP      0
QGL            0
class          0
dtype: int64


In [14]:
#stvaranje prozora, svaki prozor sadrzi 30 redaka

data_windowed = []

for df in imputed_data:
    for i in range(df.shape[0]//90):
        window = df.iloc[i*90:(i+1)*90]
        data_windowed.append(window)

In [15]:
all_windowed = pd.concat(data_windowed, keys=range(0, len(data_windowed))).reset_index()
all_windowed

,level_0,timestamp,class_code,well,instance_id,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,QGL,class
0,0,2017-09-25 01:00:30,3,14,20170925010031,2.342338e+07,1.370330e+07,56.558224,2.935319e+06,36.081306,1.625385,3.0
1,0,2017-09-25 01:00:40,3,14,20170925010031,2.342221e+07,1.370014e+07,56.564010,2.989604e+06,36.061686,1.541092,3.0
2,0,2017-09-25 01:00:50,3,14,20170925010031,2.342104e+07,1.369791e+07,56.569884,3.042945e+06,36.043620,1.513173,3.0
3,0,2017-09-25 01:01:00,3,14,20170925010031,2.342034e+07,1.369600e+07,56.576066,3.068551e+06,36.044957,1.496669,3.0
4,0,2017-09-25 01:01:10,3,14,20170925010031,2.341950e+07,1.369353e+07,56.581859,3.050824e+06,36.050174,1.411177,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4685755,52063,2018-05-13 15:33:50,2,-1,16,3.368966e+07,7.982877e+06,3.427041,4.002598e+06,31.298959,0.000000,2.0
4685756,52063,2018-05-13 15:34:00,2,-1,16,3.368966e+07,7.977187e+06,3.427143,4.002608e+06,31.241793,0.000000,2.0
4685757,52063,2018-05-13 15:34:10,2,-1,16,3.368966e+07,7.977042e+06,3.427117,4.002603e+06,31.263274,0.000000,2.0
4685758,52063,2018-05-13 15:34:20,2,-1,16,3.368966e+07,7.978292e+06,3.427058,4.002548e+06,31.201267,0.000000,2.0


In [16]:
all_windowed.rename(columns = {'level_0':'window_id'}) #maknut timestamp, ne čini se više potrebnim
all_windowed.isna().sum(axis=0)

level_0        0
timestamp      0
class_code     0
well           0
instance_id    0
P-PDG          0
P-TPT          0
T-TPT          0
P-MON-CKP      0
T-JUS-CKP      0
QGL            0
class          0
dtype: int64

In [17]:
print(all_windowed['class'].unique())
print(all_windowed['class_code'].unique())   #dinamicke klase su unistene zbog downsamplanja, nema veze ionako ih ne koristimo

[  3.           0.           5.         105.         108.
   8.         107.           7.         101.           1.
   4.         106.           6.         102.          43.71428571
   2.          29.14285714  35.33333333  84.          25.
  21.          85.          10.5         95.          42.
  65.          52.5         55.          94.5         15.
  31.5         75.          73.5         35.          63.
  45.          54.          58.          32.4         78.
  86.4         28.          10.8         75.6         38.
  43.2         68.          98.          97.2         18.
  64.8         48.          21.6         88.          30.3
  71.          40.4         61.          10.1         91.
  90.9         11.          70.7         31.          80.8
  50.5         51.          20.2         81.          60.6
  41.          74.2         36.          95.4         16.
  84.8         26.          63.6         46.          53.
  56.          31.8         76.          21.2         86.
  1

In [30]:
all_windowed.dtypes

level_0                 int64
timestamp      datetime64[ns]
class_code              int64
well                    int64
instance_id             int64
P-PDG                 float64
P-TPT                 float64
T-TPT                 float64
P-MON-CKP             float64
T-JUS-CKP             float64
QGL                   float64
class                 float64
dtype: object

In [51]:
params = {'fft_aggregated': [{'aggtype': 'centroid'},
  {'aggtype': 'variance'},
  {'aggtype': 'skew'},
  {'aggtype': 'kurtosis'}],
 'fft_coefficient': [
  {'attr': 'abs', 'coeff': 0},
  {'attr': 'abs', 'coeff': 1},
  {'attr': 'abs', 'coeff': 2},
  {'attr': 'abs', 'coeff': 3},
  {'attr': 'abs', 'coeff': 4},
  {'attr': 'abs', 'coeff': 5},
  {'attr': 'abs', 'coeff': 6},
  {'attr': 'abs', 'coeff': 7},
  {'attr': 'abs', 'coeff': 8},
  {'attr': 'abs', 'coeff': 9},
  {'attr': 'abs', 'coeff': 10},
  {'attr': 'abs', 'coeff': 11},
  {'attr': 'abs', 'coeff': 12},
  {'attr': 'abs', 'coeff': 13},
  {'attr': 'abs', 'coeff': 14},
  {'attr': 'abs', 'coeff': 15},
  {'attr': 'abs', 'coeff': 16},
  {'attr': 'abs', 'coeff': 17},
  {'attr': 'abs', 'coeff': 18},
  {'attr': 'abs', 'coeff': 19},
  {'attr': 'abs', 'coeff': 20},
  {'attr': 'abs', 'coeff': 21},
  {'attr': 'abs', 'coeff': 22},
  {'attr': 'abs', 'coeff': 23},
  {'attr': 'abs', 'coeff': 24},
  {'attr': 'abs', 'coeff': 25},
  {'attr': 'abs', 'coeff': 26},
  {'attr': 'abs', 'coeff': 27},
  {'attr': 'abs', 'coeff': 28},
  {'attr': 'abs', 'coeff': 29},
  {'attr': 'abs', 'coeff': 30},
  {'attr': 'abs', 'coeff': 31},
  {'attr': 'abs', 'coeff': 32},
  {'attr': 'abs', 'coeff': 33},
  {'attr': 'abs', 'coeff': 34},
  {'attr': 'abs', 'coeff': 35},
  {'attr': 'abs', 'coeff': 36},
  {'attr': 'abs', 'coeff': 37},
  {'attr': 'abs', 'coeff': 38},
  {'attr': 'abs', 'coeff': 39},
  {'attr': 'abs', 'coeff': 40},
  {'attr': 'abs', 'coeff': 41},
  {'attr': 'abs', 'coeff': 42},
  {'attr': 'abs', 'coeff': 43},
  {'attr': 'abs', 'coeff': 44},
  {'attr': 'abs', 'coeff': 45},
  {'attr': 'abs', 'coeff': 46},
  {'attr': 'abs', 'coeff': 47},
  {'attr': 'abs', 'coeff': 48},
  {'attr': 'abs', 'coeff': 49},
  {'attr': 'abs', 'coeff': 50},
  {'attr': 'abs', 'coeff': 51},
  {'attr': 'abs', 'coeff': 52},
  {'attr': 'abs', 'coeff': 53},
  {'attr': 'abs', 'coeff': 54},
  {'attr': 'abs', 'coeff': 55},
  {'attr': 'abs', 'coeff': 56},
  {'attr': 'abs', 'coeff': 57},
  {'attr': 'abs', 'coeff': 58},
  {'attr': 'abs', 'coeff': 59},
  {'attr': 'abs', 'coeff': 60},
  {'attr': 'abs', 'coeff': 61},
  {'attr': 'abs', 'coeff': 62},
  {'attr': 'abs', 'coeff': 63},
  {'attr': 'abs', 'coeff': 64},
  {'attr': 'abs', 'coeff': 65},
  {'attr': 'abs', 'coeff': 66},
  {'attr': 'abs', 'coeff': 67},
  {'attr': 'abs', 'coeff': 68},
  {'attr': 'abs', 'coeff': 69},
  {'attr': 'abs', 'coeff': 70},
  {'attr': 'abs', 'coeff': 71},
  {'attr': 'abs', 'coeff': 72},
  {'attr': 'abs', 'coeff': 73},
  {'attr': 'abs', 'coeff': 74},
  {'attr': 'abs', 'coeff': 75},
  {'attr': 'abs', 'coeff': 76},
  {'attr': 'abs', 'coeff': 77},
  {'attr': 'abs', 'coeff': 78},
  {'attr': 'abs', 'coeff': 79},
  {'attr': 'abs', 'coeff': 80},
  {'attr': 'abs', 'coeff': 81},
  {'attr': 'abs', 'coeff': 82},
  {'attr': 'abs', 'coeff': 83},
  {'attr': 'abs', 'coeff': 84},
  {'attr': 'abs', 'coeff': 85},
  {'attr': 'abs', 'coeff': 86},
  {'attr': 'abs', 'coeff': 87},
  {'attr': 'abs', 'coeff': 88},
  {'attr': 'abs', 'coeff': 89},
  {'attr': 'abs', 'coeff': 90},
  {'attr': 'abs', 'coeff': 91},
  {'attr': 'abs', 'coeff': 92},
  {'attr': 'abs', 'coeff': 93},
  {'attr': 'abs', 'coeff': 94},
  {'attr': 'abs', 'coeff': 95},
  {'attr': 'abs', 'coeff': 96},
  {'attr': 'abs', 'coeff': 97},
  {'attr': 'abs', 'coeff': 98},
  {'attr': 'abs', 'coeff': 99}], 
    'maximum': None,
    'minimum': None,
    'median': None,
    'quantile': [{'q': 0.1},
  {'q': 0.2},
  {'q': 0.3},
  {'q': 0.4},
  {'q': 0.6},
  {'q': 0.7},
  {'q': 0.8},
  {'q': 0.9}],
    'variation_coefficient': None,
    'mean_change': None,
    'mean_second_derivative_central': None,
    'friedrich_coefficients': [{'coeff': 0, 'm': 3, 'r': 30},
  {'coeff': 1, 'm': 3, 'r': 30},
  {'coeff': 2, 'm': 3, 'r': 30},
  {'coeff': 3, 'm': 3, 'r': 30}]}

In [60]:
test_df = all_windowed.head(900)[['timestamp', 'level_0', 'P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP', 'QGL']]
tf.extract_features(test_df, default_fc_parameters=params, column_id="level_0", column_sort="timestamp", column_kind=None, column_value=None)

Feature Extraction: 100%|███████████████████████| 10/10 [00:00<00:00, 19.38it/s]


P-PDG__fft_aggregated__aggtype_"centroid"  \
0                                   0.022270   
1                                   0.381236   
2                                   0.231358   
3                                   0.066194   
4                                   0.190839   
5                                   0.114376   
6                                   0.108277   
7                                   0.080533   
8                                   0.005513   
9                                   0.007023   

   P-PDG__fft_aggregated__aggtype_"variance"  \
0                                   0.557835   
1                                   7.678399   
2                                   5.576964   
3                                   1.629458   
4                                   4.687469   
5                                   2.789683   
6                                   2.645739   
7                                   1.967071   
8                                   0.098646   
9                                   0.152102   

   P-PDG__fft_aggregated__aggtype_"skew"  \
0                              42.812177   
1                              10.147563   
2                              13.167773   
3                              24.939578   
4                              14.496008   
5                              18.856150   
6                              19.376815   
7                              22.515683   
8                                    NaN   
9                                    NaN   

   P-PDG__fft_aggregated__aggtype_"kurtosis"  \
0                                2026.398980   
1                                 121.062514   
2                                 194.535251   
3                                 690.259544   
4                                 234.309268   
5                                 395.758017   
6                                 417.724956   
7                                 564.081086   
8                                        NaN   
9                                        NaN   

   P-PDG__fft_coefficient__attr_"abs"__coeff_0  \
0                                 2.107944e+09   
1                                 2.159340e+09   
2                                 2.142372e+09   
3                                 2.166173e+09   
4                                 2.123371e+09   
5                                 2.119547e+09   
6                                 2.154533e+09   
7                                 2.175284e+09   
8                                 2.184849e+09   
9                                 2.186182e+09   

   P-PDG__fft_coefficient__attr_"abs"__coeff_1  \
0                                 1.215102e+06   
1                                 2.654733e+07   
2                                 1.117068e+07   
3                                 3.803550e+06   
4                                 1.077189e+07   
5                                 5.586053e+06   
6                                 6.897306e+06   
7                                 5.295851e+06   
8                                 3.140883e+05   
9                                 8.863131e+05   

   P-PDG__fft_coefficient__attr_"abs"__coeff_2  \
0                                 3.317148e+05   
1                                 1.183720e+07   
2                                 4.384882e+06   
3                                 1.610291e+06   
4                                 3.326803e+06   
5                                 3.644050e+06   
6                                 2.282641e+06   
7                                 1.648705e+06   
8                                 2.318206e+05   
9                                 4.030270e+05   

   P-PDG__fft_coefficient__attr_"abs"__coeff_3  \
0                                 3.194082e+05   
1                                 6.104030e+06   
2                                 3.260944e+06   
3                                 8.932174e+05   
4                           

In [52]:
all_windowed.head()

,level_0,timestamp,class_code,well,instance_id,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,QGL,class
0,0,2017-09-25 01:00:30,3,14,20170925010031,2.342338e+07,1.370330e+07,56.558224,2.935319e+06,36.081306,1.625385,3.0
1,0,2017-09-25 01:00:40,3,14,20170925010031,2.342221e+07,1.370014e+07,56.564010,2.989604e+06,36.061686,1.541092,3.0
2,0,2017-09-25 01:00:50,3,14,20170925010031,2.342104e+07,1.369791e+07,56.569884,3.042945e+06,36.043620,1.513173,3.0
3,0,2017-09-25 01:01:00,3,14,20170925010031,2.342034e+07,1.369600e+07,56.576066,3.068551e+06,36.044957,1.496669,3.0
4,0,2017-09-25 01:01:10,3,14,20170925010031,2.341950e+07,1.369353e+07,56.581859,3.050824e+06,36.050174,1.411177,3.0


In [45]:
import tsfresh as tf
def extract_fft(df):
    param = [{'aggtype': 'centroid'},
  {'aggtype': 'variance'},
  {'aggtype': 'skew'},
  {'aggtype': 'kurtosis'}]
    return tf.feature_extraction.feature_calculators.fft_aggregated(df, [{'aggtype':'centroid'}])

In [46]:
def extract_feats(df):
    None

In [47]:

def extract_fc(df):
    param = [{'coeff': 0, 'm': 3, 'r': 30},
    {'coeff': 1, 'm': 3, 'r': 30},
    {'coeff': 2, 'm': 3, 'r': 30},
    {'coeff': 3, 'm': 3, 'r': 30}]
    return tf.feature_extraction.feature_calculators.friedrich_coefficients(df, param=param)

In [59]:
test_df = all_windowed.head(900)[['timestamp', 'level_0', 'P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP', 'QGL']]
tf.extract_features(test_df, default_fc_parameters=params, column_id="level_0", column_sort="timestamp", column_kind=None, column_value=None)

Feature Extraction: 100%|███████████████████████| 10/10 [00:00<00:00, 17.48it/s]


P-PDG__fft_aggregated__aggtype_"centroid"  \
0                                   0.022270   
1                                   0.381236   
2                                   0.231358   
3                                   0.066194   
4                                   0.190839   
5                                   0.114376   
6                                   0.108277   
7                                   0.080533   
8                                   0.005513   
9                                   0.007023   

   P-PDG__fft_aggregated__aggtype_"variance"  \
0                                   0.557835   
1                                   7.678399   
2                                   5.576964   
3                                   1.629458   
4                                   4.687469   
5                                   2.789683   
6                                   2.645739   
7                                   1.967071   
8                                   0.098646   
9                                   0.152102   

   P-PDG__fft_aggregated__aggtype_"skew"  \
0                              42.812177   
1                              10.147563   
2                              13.167773   
3                              24.939578   
4                              14.496008   
5                              18.856150   
6                              19.376815   
7                              22.515683   
8                                    NaN   
9                                    NaN   

   P-PDG__fft_aggregated__aggtype_"kurtosis"  \
0                                2026.398980   
1                                 121.062514   
2                                 194.535251   
3                                 690.259544   
4                                 234.309268   
5                                 395.758017   
6                                 417.724956   
7                                 564.081086   
8                                        NaN   
9                                        NaN   

   P-PDG__fft_coefficient__attr_"abs"__coeff_0  \
0                                 2.107944e+09   
1                                 2.159340e+09   
2                                 2.142372e+09   
3                                 2.166173e+09   
4                                 2.123371e+09   
5                                 2.119547e+09   
6                                 2.154533e+09   
7                                 2.175284e+09   
8                                 2.184849e+09   
9                                 2.186182e+09   

   P-PDG__fft_coefficient__attr_"abs"__coeff_1  \
0                                 1.215102e+06   
1                                 2.654733e+07   
2                                 1.117068e+07   
3                                 3.803550e+06   
4                                 1.077189e+07   
5                                 5.586053e+06   
6                                 6.897306e+06   
7                                 5.295851e+06   
8                                 3.140883e+05   
9                                 8.863131e+05   

   P-PDG__fft_coefficient__attr_"abs"__coeff_2  \
0                                 3.317148e+05   
1                                 1.183720e+07   
2                                 4.384882e+06   
3                                 1.610291e+06   
4                                 3.326803e+06   
5                                 3.644050e+06   
6                                 2.282641e+06   
7                                 1.648705e+06   
8                                 2.318206e+05   
9                                 4.030270e+05   

   P-PDG__fft_coefficient__attr_"abs"__coeff_3  \
0                                 3.194082e+05   
1                                 6.104030e+06   
2                                 3.260944e+06   
3                                 8.932174e+05   
4                           

In [15]:
def threeW_feature_extraction(data):
    all_feats = []
    for i, df in enumerate(data):
        if not i % 1000:
            print(i, end=' ')
        all_feats.append(my_extract_features(df))
    return all_feats

In [35]:
def my_extract_features(df):
    df = df[['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP', 'QGL']] #izbacen class

    maximum = tf.feature_extraction.feature_calculators.maximum(df).add_prefix('max_')
    minimum = tf.feature_extraction.feature_calculators.minimum(df).add_prefix('min_')
    mean_change = tf.feature_extraction.feature_calculators.mean_change(df)
    avg_scnd_derivative = tf.feature_extraction.feature_calculators.mean_second_derivative_central(df)

    mean = []
    variance = []
    skewness = []
    kurtosis = []
    abs_fft = []
    median = []
    quantile = []

    for i in ['P-PDG', 'P-TPT', 'T-TPT', 'P-MON-CKP', 'T-JUS-CKP', 'QGL']:  #izbacen class
        median.append(tf.feature_extraction.feature_calculators.median(df[i]))
        quantile.append(tf.feature_extraction.feature_calculators.quantile(df[i], 0.2))
        mean.append(list(tf.feature_extraction.feature_calculators.fft_aggregated(df[i], [{'aggtype':'centroid'}]))[0][1])
        variance.append(list(tf.feature_extraction.feature_calculators.fft_aggregated(df[i], [{'aggtype':'variance'}]))[0][1])
        skewness.append(list(tf.feature_extraction.feature_calculators.fft_aggregated(df[i], [{'aggtype':'skew'}]))[0][1])
        kurtosis.append(list(tf.feature_extraction.feature_calculators.fft_aggregated(df[i], [{'aggtype':'kurtosis'}]))[0][1])
        abs_fft.append(list(tf.feature_extraction.feature_calculators.fft_coefficient(df[i], [{'coeff':0, 'attr':'abs'}]))[0][1])

    data_max = pd.DataFrame(maximum).transpose()
    data_min = pd.DataFrame(minimum).transpose()
    data_mean = pd.DataFrame(mean).transpose().rename(columns={0:'me_P-PDG', 1:'me_P-TPT', 2:'me_T-TPT', 3:'me_P-MON-CKP', 4:'me_T-JUS-CKP', 5:'me_QGL', 6:'me_class'})
    data_variance = pd.DataFrame(variance).transpose().rename(columns={0:'var_P-PDG', 1:'var_P-TPT', 2:'var_T-TPT', 3:'var_P-MON-CKP', 4:'var_T-JUS-CKP', 5:'var_QGL', 6:'var_class'})
    data_skewness = pd.DataFrame(skewness).transpose().rename(columns={0:'sk_P-PDG', 1:'sk_P-TPT', 2:'sk_T-TPT', 3:'sk_P-MON-CKP', 4:'sk_T-JUS-CKP', 5:'sk_QGL', 6:'sk_class'})
    data_kurtosis = pd.DataFrame(kurtosis).transpose().rename(columns={0:'kt_P-PDG', 1:'kt_P-TPT', 2:'kt_T-TPT', 3:'kt_P-MON-CKP', 4:'kt_T-JUS-CKP', 5:'kt_QGL', 6:'kt_class'})
    data_abs_fft = pd.DataFrame(abs_fft).transpose().rename(columns={0:'afft_P-PDG', 1:'afft_P-TPT', 2:'afft_T-TPT', 3:'afft_P-MON-CKP', 4:'afft_T-JUS-CKP', 5:'afft_QGL', 6:'afft_class'})

    data_median = pd.DataFrame([median]).rename(columns={0:'med_P-PDG', 1:'med_P-TPT', 2:'med_T-TPT', 3:'med_P-MON-CKP', 4:'med_T-JUS-CKP', 5:'med_QGL', 6:'med_class'})
    data_quantile = pd.DataFrame([quantile]).rename(columns={0:'qtl_P-PDG', 1:'qtl_P-TPT', 2:'qtl_T-TPT', 3:'qtl_P-MON-CKP', 4:'qtl_T-JUS-CKP', 5:'qtl_QGL', 6:'qtl_class'})
    data_mean_change = pd.DataFrame(mean_change).transpose().rename(columns={0:'mc_P-PDG', 1:'mc_P-TPT', 2:'mc_T-TPT', 3:'mc_P-MON-CKP', 4:'mc_T-JUS-CKP', 5:'mc_QGL', 6:'mc_class'})
    data_asd = pd.DataFrame(avg_scnd_derivative).transpose().rename(columns={0:'asd_P-PDG', 1:'asd_P-TPT', 2:'asd_T-TPT', 3:'asd_P-MON-CKP', 4:'asd_T-JUS-CKP', 5:'asd_QGL', 6:'asd_class'})

    features = pd.concat([data_mean, data_variance, data_skewness, data_kurtosis, data_abs_fft, data_min, data_max, data_median, data_quantile, data_mean_change, data_asd], axis=1)

    #ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
    #coeff_of_variation = tf.feature_extraction.feature_calculators.variation_coefficient(df)

    #ValueError: If using all scalar values, you must pass an index
    #langevin_1 = tf.feature_extraction.feature_calculators.friedrich_coefficients(df['QGL'], [{'m':1, 'r':0.2, 'coeff':0}])
    #langevin_3 = tf.feature_extraction.feature_calculators.friedrich_coefficients(df, [{'m':3, 'r':0.2, 'coeff':1}])
    return features

    

In [36]:
#provjera jesu li uklonjene sve null vrijednosti

x = None
for df in data_windowed:
    if x is None:
        x = df.isna().sum(axis = 0)
    else:
        x += df.isna().sum(axis = 0)
print(x)

class_code     0
well           0
instance_id    0
P-PDG          0
P-TPT          0
T-TPT          0
P-MON-CKP      0
T-JUS-CKP      0
QGL            0
class          0
dtype: int64


In [38]:
df = threeW_feature_extraction(data_windowed)

0 50 100 150 200 250 300 350 400 450 500 550 600 650 700 750 

KeyboardInterrupt: 

In [39]:
df_all = pd.concat(df, ignore_index=True)

In [40]:
df_all.shape

(52064, 66)

In [41]:
df_all

,me_P-PDG,me_P-TPT,me_T-TPT,me_P-MON-CKP,me_T-JUS-CKP,me_QGL,var_P-PDG,var_P-TPT,var_T-TPT,var_P-MON-CKP,var_T-JUS-CKP,var_QGL,sk_P-PDG,sk_P-TPT,sk_T-TPT,sk_P-MON-CKP,sk_T-JUS-CKP,sk_QGL,kt_P-PDG,kt_P-TPT,kt_T-TPT,kt_P-MON-CKP,kt_T-JUS-CKP,kt_QGL,afft_P-PDG,afft_P-TPT,afft_T-TPT,afft_P-MON-CKP,afft_T-JUS-CKP,afft_QGL,min_P-PDG,min_P-TPT,min_T-TPT,min_P-MON-CKP,min_T-JUS-CKP,min_QGL,max_P-PDG,max_P-TPT,max_T-TPT,max_P-MON-CKP,max_T-JUS-CKP,max_QGL,med_P-PDG,med_P-TPT,med_T-TPT,med_P-MON-CKP,med_T-JUS-CKP,med_QGL,qtl_P-PDG,qtl_P-TPT,qtl_T-TPT,qtl_P-MON-CKP,qtl_T-JUS-CKP,qtl_QGL,mc_P-PDG,mc_P-TPT,mc_T-TPT,mc_P-MON-CKP,mc_T-JUS-CKP,mc_QGL,asd_P-PDG,asd_P-TPT,asd_T-TPT,asd_P-MON-CKP,asd_T-JUS-CKP,asd_QGL
0,0.022270,0.018428,0.083360,2.135963,0.073821,7.120489,0.557835,0.414752,2.023835,46.143544,1.730109,124.333138,42.812177,NaN,22.153243,4.106382,23.866981,1.780758,2026.398980,NaN,546.194216,20.517004,637.635810,5.667800,2.107944e+09,1.231681e+09,5120.369773,2.267932e+08,3242.213724,71.938753,23396920.0,13652481.0,56.558224,2155294.7,35.794288,0.000000,23482174.0,13726937.0,57.227508,3068551.3,36.385653,1.633145,23419108.5,13684227.50,56.780088,2442188.25,36.028804,0.542730,23400809.0,13659270.20,56.659295,2332873.14,35.883943,0.004332,660.644195,-258.086142,0.006273,-8764.319226,0.003420,-0.018263,51.714015,-5.570076,-5.152588e-05,-368.785732,0.000173,0.000479
1,0.381236,0.615268,0.444707,2.993926,0.101528,8.146690,7.678399,13.151762,7.612511,64.304260,2.453556,132.439480,10.147563,8.019541,10.626397,3.342626,20.112556,1.643413,121.062514,74.570830,133.405110,14.045253,450.362932,5.366474,2.159340e+09,1.118209e+09,4589.166574,1.474427e+08,3279.670068,46.145508,22869665.0,11424941.0,42.854535,1200219.9,35.962461,0.000000,24466544.0,13676589.0,57.110023,2175734.1,36.756317,2.533159,24284489.5,12416122.50,51.282670,1552636.30,36.482696,0.077642,23481783.2,11735792.20,46.304652,1285821.26,36.219872,0.000000,170.629213,-8274.584270,-0.022060,-9282.851685,-0.004876,0.025230,54.482955,219.090909,8.764773e-04,-13.088068,-0.000143,-0.001218
2,0.231358,1.156930,0.318461,0.833006,0.215910,3.306367,5.576964,27.207773,7.294126,19.194420,5.290091,68.853343,13.167773,5.697768,11.481575,6.895282,13.625189,3.126030,194.535251,37.603879,148.646015,54.556361,207.329525,12.636991,2.142372e+09,1.278676e+09,5038.124884,1.137424e+08,3184.876076,143.578729,23476699.0,12964007.0,53.210616,1183666.9,35.031318,1.187865,24159372.0,15119397.0,58.220286,1340303.2,35.948000,2.239493,23774610.5,14349974.50,56.087553,1270674.60,35.304474,1.534748,23586553.8,13530434.80,54.964970,1197871.42,35.121191,1.364097,7462.033708,24217.865169,-0.023100,-1490.891011,-0.010300,-0.008929,126.954545,16.062500,-1.133955e-03,-27.332955,0.000053,0.000396
3,0.066194,0.068514,0.049711,4.122415,0.374295,2.823389,1.629458,1.520922,0.966903,83.069169,8.916060,58.077568,24.939578,25.498613,29.819388,2.679864,10.374862,3.295454,690.259544,728.651124,1027.048052,9.777997,121.163840,14.138161,2.166173e+09,1.382759e+09,4728.944855,1.464918e+08,3199.667524,114.036273,23972336.0,15041208.0,51.957234,1193386.6,34.874796,0.999077,24167040.0,15507161.0,53.103125,2789044.1,36.770125,1.451167,24080461.0,15394131.00,52.532584,1473734.75,35.338850,1.269410,23978896.2,15246458.20,52.242890,1250936.32,34.949521,1.193868,-2116.426966,-1076.011236,-0.001977,13474.533708,0.018313,0.000017,-37.545455,-307.835227,3.435795e-04,226.927841,-0.000218,0.000650
4,0.190839,0.844351,0.553246,2.927248,1.085989,2.715790,4.687469,20.084129,13.237162,47.619965,24.198986,59.537106,14.496008,6.756835,8.442291,3.555787,6.058372,3.496575,234.309268,52.182283,80.670704,16.799143,42.416689,15.232295,2.123371e+09,1.258225e+09,4904.908877,2.742953e+08,2924.196237,133.354305,23360972.0,13392532.0,52.817554,1239993.0,27.484402,1.039456,23961413.0,15015627.0,56.668225,5132579.5,36.612609,1.798676,23555319.5,13833923.50,54.383077,2726934.20,33.808151,1.510960,23405253.6,13467635.00,52.917554,2058594.70,28.139058,1.272232,-6045.

In [42]:
class_codes = []
for df2 in data_windowed:
    class_codes.append(df2['class_code'].unique())
df_classes = pd.DataFrame(class_codes).rename(columns = {0:'class_code'})

In [43]:
df_classes.tail()

,class_code
52059,2
52060,2
52061,2
52062,2
52063,2


In [44]:
data_windowed[3].head(30)

,class_code,well,instance_id,P-PDG,P-TPT,T-TPT,P-MON-CKP,T-JUS-CKP,QGL,class
timestamp,,,,,,,,,,
2017-09-25 01:45:30,3,14,20170925010031,24161488.0,15136973.0,53.103125,1193386.6,35.026269,1.449690,3.0
2017-09-25 01:45:40,3,14,20170925010031,24163605.0,15151287.0,53.023225,1198757.8,35.021220,1.378607,3.0
2017-09-25 01:45:50,3,14,20170925010031,24165549.0,15171095.0,52.920851,1203075.9,35.016170,1.298327,3.0
2017-09-25 01:46:00,3,14,20170925010031,24166095.0,15184504.0,52.855164,1212416.9,35.011121,1.270610,3.0
2017-09-25 01:46:10,3,14,20170925010031,24166162.0,15199568.0,52.799512,1214917.0,35.006072,1.248529,3.0
2017-09-25 01:46:20,3,14,20170925010031,24166829.0,15216823.0,52.744863,1216250.2,35.001024,1.216666,3.0
2017-09-25 01:46:30,3,14,20170925010031,24167040.0,15232434.0,52.689845,1215462.4,34.995974,1.251921,3.0
2017-09-25 01:46:40,3,14,20170925010031,24166172.0,15245427.0,52.652402,1216113.8,34.990925,1.284521,3.0
2017-09-25 01:46:50,3,14,20170925010031,24165565.0,15259790.0,52.637630,1216659.0,34.985876,1.237393,3.0


In [45]:
df_all.head()

,me_P-PDG,me_P-TPT,me_T-TPT,me_P-MON-CKP,me_T-JUS-CKP,me_QGL,var_P-PDG,var_P-TPT,var_T-TPT,var_P-MON-CKP,var_T-JUS-CKP,var_QGL,sk_P-PDG,sk_P-TPT,sk_T-TPT,sk_P-MON-CKP,sk_T-JUS-CKP,sk_QGL,kt_P-PDG,kt_P-TPT,kt_T-TPT,kt_P-MON-CKP,kt_T-JUS-CKP,kt_QGL,afft_P-PDG,afft_P-TPT,afft_T-TPT,afft_P-MON-CKP,afft_T-JUS-CKP,afft_QGL,min_P-PDG,min_P-TPT,min_T-TPT,min_P-MON-CKP,min_T-JUS-CKP,min_QGL,max_P-PDG,max_P-TPT,max_T-TPT,max_P-MON-CKP,max_T-JUS-CKP,max_QGL,med_P-PDG,med_P-TPT,med_T-TPT,med_P-MON-CKP,med_T-JUS-CKP,med_QGL,qtl_P-PDG,qtl_P-TPT,qtl_T-TPT,qtl_P-MON-CKP,qtl_T-JUS-CKP,qtl_QGL,mc_P-PDG,mc_P-TPT,mc_T-TPT,mc_P-MON-CKP,mc_T-JUS-CKP,mc_QGL,asd_P-PDG,asd_P-TPT,asd_T-TPT,asd_P-MON-CKP,asd_T-JUS-CKP,asd_QGL
0,0.022270,0.018428,0.083360,2.135963,0.073821,7.120489,0.557835,0.414752,2.023835,46.143544,1.730109,124.333138,42.812177,NaN,22.153243,4.106382,23.866981,1.780758,2026.398980,NaN,546.194216,20.517004,637.635810,5.667800,2.107944e+09,1.231681e+09,5120.369773,2.267932e+08,3242.213724,71.938753,23396920.0,13652481.0,56.558224,2155294.7,35.794288,0.000000,23482174.0,13726937.0,57.227508,3068551.3,36.385653,1.633145,23419108.5,13684227.5,56.780088,2442188.25,36.028804,0.542730,23400809.0,13659270.2,56.659295,2332873.14,35.883943,0.004332,660.644195,-258.086142,0.006273,-8764.319226,0.003420,-0.018263,51.714015,-5.570076,-0.000052,-368.785732,0.000173,0.000479
1,0.381236,0.615268,0.444707,2.993926,0.101528,8.146690,7.678399,13.151762,7.612511,64.304260,2.453556,132.439480,10.147563,8.019541,10.626397,3.342626,20.112556,1.643413,121.062514,74.570830,133.405110,14.045253,450.362932,5.366474,2.159340e+09,1.118209e+09,4589.166574,1.474427e+08,3279.670068,46.145508,22869665.0,11424941.0,42.854535,1200219.9,35.962461,0.000000,24466544.0,13676589.0,57.110023,2175734.1,36.756317,2.533159,24284489.5,12416122.5,51.282670,1552636.30,36.482696,0.077642,23481783.2,11735792.2,46.304652,1285821.26,36.219872,0.000000,170.629213,-8274.584270,-0.022060,-9282.851685,-0.004876,0.025230,54.482955,219.090909,0.000876,-13.088068,-0.000143,-0.001218
2,0.231358,1.156930,0.318461,0.833006,0.215910,3.306367,5.576964,27.207773,7.294126,19.194420,5.290091,68.853343,13.167773,5.697768,11.481575,6.895282,13.625189,3.126030,194.535251,37.603879,148.646015,54.556361,207.329525,12.636991,2.142372e+09,1.278676e+09,5038.124884,1.137424e+08,3184.876076,143.578729,23476699.0,12964007.0,53.210616,1183666.9,35.031318,1.187865,24159372.0,15119397.0,58.220286,1340303.2,35.948000,2.239493,23774610.5,14349974.5,56.087553,1270674.60,35.304474,1.534748,23586553.8,13530434.8,54.964970,1197871.42,35.121191,1.364097,7462.033708,24217.865169,-0.023100,-1490.891011,-0.010300,-0.008929,126.954545,16.062500,-0.001134,-27.332955,0.000053,0.000396
3,0.066194,0.068514,0.049711,4.122415,0.374295,2.823389,1.629458,1.520922,0.966903,83.069169,8.916060,58.077568,24.939578,25.498613,29.819388,2.679864,10.374862,3.295454,690.259544,728.651124,1027.048052,9.777997,121.163840,14.138161,2.166173e+09,1.382759e+09,4728.944855,1.464918e+08,3199.667524,114.036273,23972336.0,15041208.0,51.957234,1193386.6,34.874796,0.999077,24167040.0,15507161.0,53.103125,2789044.1,36.770125,1.451167,24080461.0,15394131.0,52.532584,1473734.75,35.338850,1.269410,23978896.2,15246458.2,52.242890,1250936.32,34.949521,1.193868,-2116.426966,-1076.011236,-0.001977,13474.533708,0.018313,0.000017,-37.545455,-307.835227,0.000344,226.927841,-0.000218,0.000650
4,0.190839,0.844351,0.553246,2.927248,1.085989,2.715790,4.687469,20.084129,13.237162,47.619965,24.198986,59.537106,14.496008,6.756835,8.442291,3.555787,6.058372,3.496575,234.309268,52.182283,80.670704,16.799143,42.416689,15.232295,2.123371e+09,1.258225e+09,4904.908877,2.742953e+08,2924.196237,133.354305,23360972.0,13392532.0,52.817554,1239993.0,27.484402,1.039456,23961413.0,15015627.0,56.668225,5132579.5,36.612609,1.798676,23555319.5,13833923.5,54.383077,2726934.20,33.808151,1.510960,23405253.6,13467635.0,52.917554,2058594.70,28.139058,1.272232,-6045.359551,-16927.426966,0.042

In [52]:
na_count = df_all.isna().sum(axis=0)[:][:]
for n in na_count:
    print(n)

8655
64
6514
1200
1819
50588
8655
64
6514
1200
1819
50588
37544
30737
41741
31206
28769
50596
37544
30737
41741
31206
28769
50596
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [29]:
df_all = df_all.fillna(df_all.mean())

In [30]:
df_all.isna().sum(axis=0)

me_P-PDG         0
me_P-TPT         0
me_T-TPT         0
me_P-MON-CKP     0
me_T-JUS-CKP     0
                ..
asd_T-TPT        0
asd_P-MON-CKP    0
asd_T-JUS-CKP    0
asd_QGL          0
asd_class        0
Length: 77, dtype: int64

NameError: name 'df_all' is not defined